In [ ]:
# Import lib

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re
import random

C:\Users\msi\anaconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set up device utilize device gpu 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


Device: NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
# load the model and the tokenizer

In [3]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

C:\Users\msi\anaconda3\envs\nlp_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\msi\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
# Extra padding to avoid warnings

In [5]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

In [ ]:
# Moving the model to the device 

In [6]:
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
# Examples retrieved from the dataset 
examples = [
    # POSITIVE
    (
        "Think wonderful way spend time hot summer weekend sit air condition theater watch lighthearted comedy plot simplistic dialogue witty character likable even well bread suspect serial killer may disappoint realize match point 2 risk addiction think proof woody allen still fully control style many us grow lovethis i d laugh one woodys comedy year dare say decade I ve never impress scarlet johanson manage tone sexy image jump right average spirited young womanthis may crown jewel career witty devil wear prada interesting superman great comedy go see friend",
        "Positive"
    ),
    (
        "Wonderful little production filming technique unassume oldtimebbc fashion give comfort sometimes discomforte sense realism entire piece actor extremely well choose michael sheen get polari voice pat truly see seamless editing guide reference williams diary entry well worth watching terrificly write perform piece masterful production one great master comedy life realism really come home little thing fantasy guard rather use traditional dream technique remain solid disappear play knowledge sense particularly scene concern orton halliwell set particularly flat halliwell mural decorate every surface terribly well do",
        "Positive"
    ),
    (
        "One reviewer mention watch 1 oz episode you ll hook right exactly happen methe first thing strike oz brutality unflinche scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use wordit call oz nickname give oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inward privacy high agenda em city home manyaryan muslim gangstas latinos christians italians irish moreso scuffle death stare dodgy dealing shady agreement never far awayi would say main appeal show due fact go show would not dare forget pretty picture paint mainstream audience forget charm forget romanceoz do not mess around first episode ever see strike nasty surreal could not say ready watch develop taste oz get accustomed high level graphic violence violence injustice crook guard who ll sell nickel inmate who ll kill order get away well mannered middle class inmate turn prison bitch due lack street skill prison experience watch oz may become comfortable uncomfortable viewingthat get touch dark side",
        "Positive"
    ),
    # NEGATIVE
    (
        "Basically there s family little boy jake think there s zombie closet parent fight timethis movie slow soap opera suddenly jake decide become rambo kill zombieok first you re go make film must decide thriller drama drama movie watchable parent divorce argue like real life jake closet totally ruin film expect see boogeyman similar movie instead watch drama meaningless thriller spots3 10 well play parent descent dialog shot jake ignore show",
        "Negative"
    ),
    (
        "Amazing fresh innovative idea 70 first air first 7 8 year brilliant thing drop 1990 show really funny anymore continue decline complete waste time todayit truly disgraceful far show fall write painfully bad performance almost bad mildly entertaining respite guesthost show probably would not still air find hard believe creator handselecte original cast also choose band hack follow one recognize brilliance see fit replace mediocrity feel must give 2 star respect original cast make show huge success show awful can not believe still air",
        "Negative"
    )
]

In [8]:
# Prompt builder
def create_prompt(review, num_shots=1):
    selected = random.sample(examples, num_shots)
    prompt = ""
    for ex_text, label in selected:
        prompt += f"Review: {ex_text}\nSentiment: {label}\n\n"
    prompt += f"Review: {review}\nSentiment:"
    return prompt


In [9]:
# Classifier
def classify_sentiment(review, num_shots=1, max_new_tokens=10):
    prompt = create_prompt(review, num_shots)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    match = re.search(r"Sentiment:\s*(Positive|Negative)", generated_text, re.IGNORECASE)
    return match.group(1).capitalize() if match else "Invalid Output"


In [10]:
# Main interactive loop
if __name__ == "__main__":
    review_input = input("Enter a new review to classify:\n")
    for shots in [1, 2, 3]:
        result = classify_sentiment(review_input, num_shots=shots)
        print(f"{shots}-shot Prediction: {result}")

Enter a new review to classify:
 the movie sucks 


1-shot Prediction: Positive
2-shot Prediction: Positive
3-shot Prediction: Negative
